In [1]:
from boosting_model.processing.load_data import load_dataset,save_pipeline
from boosting_model.processing.validation import validate_input
from boosting_model import pipeline
from boosting_model.config.core import config
from xgboost import XGBRegressor
import joblib
from boosting_model.config import core
from sklearn.pipeline import make_pipeline


In [2]:
x = load_dataset(filename=config.app_config.training_x_file)
y = load_dataset(filename=config.app_config.training_y_file)

In [3]:
x.rename(columns=config.model_config.rename,inplace=True)

In [6]:
path = core.TRAINED_MODEL_DIR/ config.app_config.saved_model_name

In [7]:
model = pipeline.model_maker(model=XGBRegressor())
model.load_model(path)
    

'loaded model'

In [8]:
model

model_maker(model=XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, early_stopping_rounds=None,
                               enable_categorical=False, eval_metric=None,
                               gamma=0, gpu_id=-1, grow_policy='depthwise',
                               importance_type=None, interaction_constraints='',
                               learning_rate=0.300000012, max_bin=256,
                               max_cat_to_onehot=4, max_delta_step=0,
                               max_depth=10, max_leaves=0, min_child_weight=1,
                               missing=nan, monotone_constraints='()',
                               n_estimators=100, n_jobs=0, num_parallel_tree=1,
                               predictor='auto', random_state=0, reg_alpha=2,
                               reg_lambda=3, ...))

In [9]:
p = pipeline.pipe.fit(x)


In [10]:
xx = p.transform(x)
xx.head()

,online_order,book_table,votes,approx_cost,listed_in_type_Buffet,listed_in_type_Cafes,listed_in_type_Delivery,listed_in_type_Desserts,listed_in_type_Dine-out,listed_in_type_Drinks & nightlife,...,veg,visit,visited,want,went,worth,xfã,xs,xã,zomato
38765,0,0,0.932101,1.107742,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.033003,0.0,0.043458,0.000000,0.000000,0.0,0.0,0.759688,0.00000
24221,1,0,0.553880,0.566266,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.459509,0.0,0.0,0.000000,0.18455
34893,1,0,0.830707,-0.282696,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.00000
16478,1,0,0.057207,-2.378461,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.394047,0.000000,0.0,0.0,0.000000,0.00000
16607,1,0,-0.916078,-0.282696,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.204625,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.00000


In [11]:

#joblib.dump(p,'best_model/only_pipeline2.pkl')

In [12]:
#joblib.load('best_model/only_pipeline2.pkl')

In [15]:
final_pipe = make_pipeline(p,model)

In [16]:
print(final_pipe.predict(x.iloc[:10]))

[3.5823956 3.6694782 3.880002  3.92301   3.5107567 3.3534896 3.5878024
 3.8492756 3.4758792 3.8052106]


In [18]:
#joblib.dump(final_pipe,'best_model/final_pipeline2.pkl')

In [4]:
final_pipe = core.TRAINED_MODEL_DIR/ config.app_config.pipeline_name
print(final_pipe)

/Users/rahul/anacond/anaconda3/lib/python3.9/site-packages/boosting_model-0.0.0-py3.9.egg/boosting_model/best_model/final_pipeline2.pkl


In [5]:
r = joblib.load(final_pipe)

In [6]:
r.steps

[('pipeline',
  Pipeline(steps=[('drop feature', featuresDrop(variables=['location', 'name'])),
                  ('functional',
                   sklearnTransformerWrapper(transformer=FunctionTransformer(func=<function transform_log at 0x7f9a247da160>),
                                             variable=['votes', 'approx_cost'])),
                  ('scale',
                   sklearnTransformerWrapper(transformer=StandardScaler(),
                                             variable=['votes', 'approx_cost'])),
                  ('onehot',
                   sklearnOne(transformer=OneHotEncoder(handle_unknown='ignore',
                                                        sparse=False),
                              variable=['listed_in_type', 'rest_type',
                                        'listed_in_city'])),
                  ('countVector', sklearnCountVector(variable='cuisines')),
                  ('tfidf', reviewsTransform(variable='reviews_list'))])),
 ('model_make

In [7]:
r.predict(x.iloc[:10])

array([3.5823956, 3.6694782, 3.880002 , 3.92301  , 3.5107567, 3.3534896,
       3.5878024, 3.8492756, 3.4758792, 3.8052106], dtype=float32)

In [8]:
x = load_dataset(filename=config.app_config.training_x_file)
y = load_dataset(filename=config.app_config.training_y_file)

In [10]:
x,e = validate_input(input_data=x)

/Users/rahul/anacond/anaconda3/lib/python3.9/site-packages/boosting_model-0.0.0-py3.9.egg/boosting_model/processing/validation.py:37: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  schema.load(input_data.replace(


In [11]:
x.head()

,name,online_order,book_table,votes,location,rest_type,cuisines,approx_cost,reviews_list,listed_in_type,listed_in_city
38765,WYT RestroPub,0,0,406,MG Road,Pub,"North Indian, Chinese",1000,"[('Rated 1.0', ""RATED\n It's just a pocket fr...",Dine-out,Lavelle Road
24221,Olive Garden,1,0,214,Kalyan Nagar,Casual Dining,"Arabian, North Indian, Chinese, Beverages",700,"[('Rated 4.0', 'RATED\n Loved the al faham ch...",Delivery,Kalyan Nagar
34893,Hotel Manu,1,0,342,HSR,Quick Bites,"South Indian, Kerala, Chinese, North Indian",400,"[('Rated 4.0', 'RATED\n I have ordered a mala...",Delivery,Koramangala 7th Block
16478,Mumbai Kulfi,1,0,92,HSR,Dessert Parlor,Ice Cream,100,"[('Rated 4.0', 'RATED\n went here on sunday e...",Delivery,HSR
16607,Bellandur Kitchen,1,0,17,Bellandur,Quick Bites,"North Indian, Chinese",400,"[('Rated 5.0', 'RATED\n Egg omlet is damn tas...",Delivery,HSR


In [12]:
r.predict(x.iloc[:10])

array([3.5823956, 3.6694782, 3.880002 , 3.92301  , 3.5107567, 3.3534896,
       3.5878024, 3.8492756, 3.4758792, 3.8052106], dtype=float32)

In [25]:
x = load_dataset(filename=config.app_config.training_x_file)
y = load_dataset(filename=config.app_config.training_y_file)

x.reset_index(drop=True,inplace=True)
y.index = x.index

xx = x.sample(100)


In [26]:
yy = y.iloc[xx.index]

In [24]:
from sklearn.metrics import r2_score

In [29]:
xx,e = validate_input(input_data=xx)
yhat = r.predict(xx)

/Users/rahul/anacond/anaconda3/lib/python3.9/site-packages/boosting_model-0.0.0-py3.9.egg/boosting_model/processing/validation.py:37: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  schema.load(input_data.replace(


In [30]:
r2_score(yy,yhat)

0.8664856895231372

In [31]:
# xx.to_csv('data/x_train.csv')
# yy.to_csv('data/y_train.csv')